In [39]:
#IMPORTING ALL REQUIRED LIBRARIES
import os
import speech_recognition as sr
from selenium import webdriver
import webbrowser
from playsound import playsound
from googlesearch import search
from tkinter import *
import time
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
import urllib.request
import random

In [40]:
recognizer = sr.Recognizer()
microphone = sr.Microphone(device_index=2)
first_run = True
festival = None
contact = None

In [46]:
def recordSpeech(returnVal=False):
    global recognizer
    global microphone
    global first_run
    if(first_run):
        playsound("data/joanna/firstRun.mp3")
        with microphone as source:
            recognizer.adjust_for_ambient_noise(source)
        playsound("data/joanna/firstRunC.mp3")
        first_run = False
    with microphone as source:
        if not(returnVal):
            playsound("data/joanna/helpYou.mp3")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
    try:
        recogText = recognizer.recognize_google(audio)
        if returnVal:
            return recogText.lower()
        else:
            goAhead(recogText.lower()) 
            
    except Exception as e:
        print(e)
        playsound("data/joanna/couldNRe.mp3")

In [47]:
def goAhead(recogText):
    #CONVERTING SPEECH TO TEXT AND SAVING IT
    if "speech to text" in recogText.lower():
        playsound("data/joanna/speechTT.mp3")
        speech = recordSpeech(True)
        f = open("RecognisedText.txt","w+")
        f.write(speech)
        f.close()
        os.startfile("RecognisedText.txt")
        playsound("data/joanna/hereI.mp3")
    #OPENING YOUTUBE
    if "open youtube" in recogText:
        playsound("data/joanna/youtube.mp3")
        webbrowser.open("https://www.youtube.com")
    #OPENING SPOTIFY APP (WINDOWS 10 SPECIFIC)
    if os.name == "nt":
        if "open spotify" in recogText:
            playsound("data/joanna/spotify.mp3")
            os.system("spotify")
    else:
        playsound("data/joanna/osN.mp3")
    #PLAYING A SONG ON YOUTUBE
    if "play a song" in recogText:
        playsound("data/joanna/songPlay.mp3")
        songName = recordSpeech(True)+" youtube"
        for x in search(songName,tld="co.in",num=1,stop=1,pause=2):
            webbrowser.open(x)
        playsound("data/joanna/hereI.mp3")
    #SEARCHING GOOGLE
    if "google" in recogText or "search" in recogText:
        playsound("data/joanna/google.mp3")
        search = recordSpeech(True)
        webbrowser.open("https://www.google.com/search?q="+search.replace(" ","+"))
        playsound("data/joanna/hereI.mp3")
    if "wish" in recogText:
        global entry
        global submit
        global festival
        playsound("data/joanna/festival.mp3")
        festival = recordSpeech(True)
        entry.place(relx=0,rely=0.9,anchor=NW)
        submit.place(relx=1,rely=0.9,anchor=NE)
        submit["command"] = contactGetData
        playsound("data/joanna/festivalMid.mp3")

In [48]:
def contactGetData():
    global entry
    global submit
    global entryVar
    global contact
    global window
    temp = entry.get()
    contact = temp.split(",")
    entryVar.set("")
    print(contact)
    entry.destroy()
    submit.destroy()
    window.update()
    sendMessage()
    playsound("data/joanna/festivalEnd.mp3")
    

In [1]:
def sendMessage():
    global contact
    global festival
    options = webdriver.ChromeOptions()
    options.add_argument(r'user-data-dir=./driver/data')
    chromeDriver = webdriver.Chrome(executable_path=r'data/chromedriver.exe', options=options)

    chromeDriver.get('https://images.google.com')


    search = chromeDriver.find_element_by_name('q')
    search.send_keys(festival+ Keys.ENTER)
    elem1 = chromeDriver.find_element_by_id('islmp')
    sub = elem1.find_elements_by_tag_name('img')
    rN = random.randint(0,20)
    imgLink = sub[rN].get_attribute('src')
    urllib.request.urlretrieve(imgLink, "data/images/temp.jpg")

    wait = WebDriverWait(chromeDriver,600)
    chromeDriver.get('https://web.whatsapp.com/')
    for userName in contact:
        searchXPath = '//*[@id="side"]/div[1]/div/label/div/div[2]'
        search = wait.until(EC.element_to_be_clickable((By.XPATH,searchXPath)))
        search.send_keys(userName)
        userXPath = f'//span[@title="{userName}"]'
        user = wait.until(EC.element_to_be_clickable((By.XPATH,userXPath)))
        time.sleep(2)
        user.click()
        chromeDriver.find_element_by_css_selector("span[data-icon='clip']").click();
        chromeDriver.find_element_by_css_selector("input[type='file']").send_keys(os.getcwd()+"\\data\\images\\temp.jpg");
        wait.until(EC.presence_of_element_located((By.CSS_SELECTOR,"span[data-icon='send']"))).click()
        inputXPath = '//*[@id="main"]/footer/div[1]/div[2]/div/div[2]'
        inputBox = wait.until(EC.element_to_be_clickable((By.XPATH,inputXPath)))
        inputBox.send_keys(festival+Keys.ENTER)


In [50]:
#APPLICATION WINDOW FOR THE SOFTWARE
window = Tk()
window.geometry("300x300")
iconImage = PhotoImage(file="data/images/icon.png",master=window)
pButton = Button(window,image=iconImage,relief=RIDGE,bd=0)
pButton.place(relx=0.5,rely=0.5,anchor=CENTER)
pButton["command"] = lambda : recordSpeech()
entryVar = StringVar()
entry = Entry(window,justify=CENTER,textvariable=entryVar,font=("Calibri",15))
submit = Button(window,text="SUBMIT")
window.mainloop()

['Momsy', 'Anjali', 'Bot']
